In [20]:
import pandas as pd  # pyright: ignore[reportMissingImports]
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  # pyright: ignore[reportMissingModuleSource]
import plotly.express as px  # pyright: ignore[reportMissingImports]
import plotly.graph_objects as go  # pyright: ignore[reportMissingImports]
import plotly.io as pio  # pyright: ignore[reportMissingImports]


In [21]:
df = pd.read_csv("dengu_out_break.csv")
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41997 entries, 0 to 41996
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   adm_0_name                    41997 non-null  object 
 1   adm_1_name                    39799 non-null  object 
 2   adm_2_name                    2802 non-null   object 
 3   full_name                     41997 non-null  object 
 4   ISO_A0                        41997 non-null  object 
 5   FAO_GAUL_code                 41997 non-null  int64  
 6   RNE_iso_code                  41997 non-null  object 
 7   IBGE_code                     0 non-null      float64
 8   calendar_start_date           41997 non-null  object 
 9   calendar_end_date             41997 non-null  object 
 10  Year                          41997 non-null  int64  
 11  dengue_total                  41997 non-null  int64  
 12  case_definition_standardised  41997 non-null  object 
 13  S

/tmp/ipykernel_8406/1708877015.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dengu_out_break.csv")


In [22]:
df.head()

,adm_0_name,adm_1_name,adm_2_name,full_name,ISO_A0,FAO_GAUL_code,RNE_iso_code,IBGE_code,calendar_start_date,calendar_end_date,Year,dengue_total,case_definition_standardised,S_res,T_res,UUID,region
0,BANGLADESH,NaN,NaN,BANGLADESH,BGD,23,BGD,NaN,2015-12-01,2015-12-31,2015,75,Confirmed,Admin0,Month,MOH-BGD-20002021-Y01-01,SEARO
1,BANGLADESH,NaN,NaN,BANGLADESH,BGD,23,BGD,NaN,2017-09-01,2017-09-30,2017,430,Confirmed,Admin0,Month,MOH-BGD-20002021-Y01-01,SEARO
2,BANGLADESH,NaN,NaN,BANGLADESH,BGD,23,BGD,NaN,2016-03-01,2016-03-31,2016,17,Confirmed,Admin0,Month,MOH-BGD-20002021-Y01-01,SEARO
3,BANGLADESH,NaN,NaN,BANGLADESH,BGD,23,BGD,NaN,2016-04-01,2016-04-30,2016,38,Confirmed,Admin0,Month,MOH-BGD-20002021-Y01-01,SEARO
4,BANGLADESH,NaN,NaN,BANGLADESH,BGD,23,BGD,NaN,2016-01-01,2016-01-31,2016,13,Confirmed,Admin0,Month,MOH-BGD-20002021-Y01-01,SEARO


# Dengue Outbreak Prediction: Next 7 Days

## Required Features for 7-Day Prediction

For predicting dengue outbreaks in the next 7 days, we need the following features:

### 1. **Weather/Climate Features** (Most Critical)
- **Temperature**: Mean, min, max daily temperature
  - Optimal range: 20-30°C for mosquito breeding
  - Affects virus replication in mosquitoes
- **Rainfall/Precipitation**: Daily rainfall amount
  - Creates breeding sites (standing water)
  - Lag effect: 2-4 weeks after heavy rainfall
- **Humidity**: Relative humidity percentage
  - Affects mosquito survival and activity
- **Wind Speed**: Can affect mosquito dispersal

### 2. **Historical Case Data** (Temporal Features)
- **Lag Features**: Cases from previous days/weeks
  - 7-day, 14-day, 21-day, 30-day rolling averages
  - Recent case trends (increasing/decreasing)
- **Seasonal Patterns**: 
  - Month, day of year
  - Historical averages for same period
- **Outbreak History**: 
  - Previous outbreak intensity in same region/season

### 3. **Population & Immunity Features**
- **Population Density**: Urban vs rural
- **Previous Exposure**: 
  - Cases in last 6-12 months (proxy for immunity)
  - Seroprevalence data (if available)
- **Population Mobility**: Movement patterns

### 4. **Geographic Features**
- **Location**: Country, region, administrative levels
- **Urbanization**: Urban vs rural classification
- **Elevation**: Affects temperature and mosquito habitat

### 5. **Vector-Related Features** (If Available)
- **Mosquito Density**: Aedes aegypti/Aedes albopictus indices
- **Breeding Site Density**: Container index, Breteau index


## Data Sources for Feature Collection

### Weather/Climate Data Sources:

1. **OpenWeatherMap API** (Free tier available)
   - Current and forecast weather
   - Historical data (paid)
   - https://openweathermap.org/api

2. **NOAA Climate Data** (Free)
   - Global historical weather data
   - https://www.ncei.noaa.gov/

3. **World Bank Climate Data** (Free)
   - Historical climate data by country
   - https://climateknowledgeportal.worldbank.org/

4. **WeatherAPI.com** (Free tier)
   - Current, forecast, and historical data
   - https://www.weatherapi.com/

5. **ERA5 Reanalysis Data** (Free, via CDS)
   - High-resolution climate data
   - https://cds.climate.copernicus.eu/

### Population & Demographic Data:

1. **World Bank Open Data** (Free)
   - Population density, demographics
   - https://data.worldbank.org/

2. **UN Population Division** (Free)
   - Population estimates and projections
   - https://population.un.org/

### Health/Immunity Data:

1. **WHO Global Health Observatory** (Free)
   - Health statistics, vaccination coverage
   - https://www.who.int/data/gho

2. **DHS Program** (Free, registration required)
   - Demographic and health surveys
   - Seroprevalence data
   - https://dhsprogram.com/

### Vector Surveillance Data:

1. **Local Health Departments**
   - Mosquito surveillance data
   - Breeding site indices

2. **WHO Vector Control** (Free)
   - Regional vector control reports
   - https://www.who.int/teams/control-of-neglected-tropical-diseases/vector-ecology-and-management


In [23]:
# Example: Feature Engineering for 7-Day Prediction

# First, let's prepare the existing data
df['calendar_start_date'] = pd.to_datetime(df['calendar_start_date'])
df['calendar_end_date'] = pd.to_datetime(df['calendar_end_date'])

# Extract temporal features from existing data
df['month'] = df['calendar_start_date'].dt.month
df['day_of_year'] = df['calendar_start_date'].dt.dayofyear
df['year'] = df['calendar_start_date'].dt.year
df['quarter'] = df['calendar_start_date'].dt.quarter

# Sort by location and date for lag features
df = df.sort_values(['ISO_A0', 'adm_1_name', 'calendar_start_date'])

print("Temporal features extracted!")
print(f"Date range: {df['calendar_start_date'].min()} to {df['calendar_start_date'].max()}")
print(f"Unique countries: {df['ISO_A0'].nunique()}")
print(f"Countries: {df['ISO_A0'].unique()[:10]}")


Temporal features extracted!
Date range: 1958-01-01 00:00:00 to 2025-03-01 00:00:00
Unique countries: 10
Countries: ['BGD' 'BTN' 'IDN' 'IND' 'LKA' 'MDV' 'MMR' 'NPL' 'THA' 'TLS']


In [24]:
# Example: Creating lag features for prediction
# This shows how to create features from historical case data

def create_lag_features(df, location_col='ISO_A0', date_col='calendar_start_date', 
                       case_col='dengue_total', lags=[7, 14, 21, 30, 60, 90]):
    """
    Create lag features for dengue cases
    """
    df_lagged = df.copy()
    df_lagged = df_lagged.sort_values([location_col, date_col])
    
    for lag in lags:
        df_lagged[f'cases_lag_{lag}d'] = df_lagged.groupby(location_col)[case_col].shift(lag)
        df_lagged[f'cases_rolling_mean_{lag}d'] = df_lagged.groupby(location_col)[case_col].transform(
            lambda x: x.shift(1).rolling(window=lag, min_periods=1).mean()
        )
    
    # Create trend features
    df_lagged['cases_7d_avg'] = df_lagged.groupby(location_col)[case_col].transform(
        lambda x: x.shift(1).rolling(window=7, min_periods=1).mean()
    )
    df_lagged['cases_30d_avg'] = df_lagged.groupby(location_col)[case_col].transform(
        lambda x: x.shift(1).rolling(window=30, min_periods=1).mean()
    )
    df_lagged['cases_trend'] = df_lagged['cases_7d_avg'] - df_lagged['cases_30d_avg']
    
    return df_lagged

# Apply lag feature creation
df_with_lags = create_lag_features(df)

print("Lag features created!")
print(df_with_lags[['ISO_A0', 'calendar_start_date', 'dengue_total', 
                    'cases_lag_7d', 'cases_7d_avg', 'cases_trend']].head(20))


Lag features created!
   ISO_A0 calendar_start_date  dengue_total  cases_lag_7d  cases_7d_avg  \
27    BGD          1980-01-01             4           NaN           NaN   
28    BGD          1985-01-01             0           NaN      4.000000   
29    BGD          1986-01-01             0           NaN      2.000000   
30    BGD          1987-01-01             0           NaN      1.333333   
31    BGD          1988-01-01             0           NaN      1.000000   
32    BGD          1989-01-01             0           NaN      0.800000   
46    BGD          1990-01-01             0           NaN      0.666667   
47    BGD          1991-01-01             0           4.0      0.571429   
48    BGD          1992-01-01             0           0.0      0.000000   
36    BGD          1993-01-01             0           0.0      0.000000   
37    BGD          1994-01-01             0           0.0      0.000000   
38    BGD          1995-01-01             0           0.0      0.000000   
39 

In [25]:
# Example: Function to fetch weather data (template)
# You'll need to install: pip install requests

import requests
from datetime import datetime, timedelta

def fetch_weather_data_owm(location_name, lat, lon, api_key, start_date, end_date):
    """
    Fetch weather data from OpenWeatherMap API
    
    Note: Free tier has limited historical data access.
    For production, consider paid plans or other sources.
    """
    # For current/forecast data (free tier)
    current_url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"
    
    # For historical data, you need One Call API 3.0 (paid)
    # historical_url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={timestamp}&appid={api_key}"
    
    try:
        response = requests.get(current_url)
        data = response.json()
        
        weather_features = {
            'temperature': data['main']['temp'],
            'temp_min': data['main']['temp_min'],
            'temp_max': data['main']['temp_max'],
            'humidity': data['main']['humidity'],
            'pressure': data['main']['pressure'],
            'wind_speed': data['wind']['speed'] if 'wind' in data else None,
            'rainfall': data.get('rain', {}).get('1h', 0) if 'rain' in data else 0
        }
        return weather_features
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return None

# Example usage (you'll need to get API key from openweathermap.org)
# weather = fetch_weather_data_owm("Dhaka", 23.8103, 90.4125, "YOUR_API_KEY", None, None)
# print(weather)


In [26]:
# Alternative: Using wbpy for World Bank climate data (Free, no API key needed)
# Install: pip install wbpy

# Example function to get climate data
def get_world_bank_climate_data(country_code, indicator='temperature'):
    """
    Fetch climate data from World Bank
    
    Indicators:
    - 'temperature': Average temperature
    - 'precipitation': Average precipitation
    """
    try:
        from wbpy import ClimateAPI
        api = ClimateAPI("YOUR_EMAIL")  # Email for identification
        
        # Get data for a country
        data = api.get_indicators(country_code, [indicator])
        return data
    except Exception as e:
        print(f"Error: {e}")
        print("Note: Install wbpy with: pip install wbpy")
        return None

# For Bangladesh (BGD)
# temp_data = get_world_bank_climate_data("BGD", "temperature")


In [27]:
# Example: Creating a feature engineering pipeline for 7-day prediction

def prepare_features_for_prediction(df, target_location='BGD', prediction_date=None):
    """
    Prepare all features needed for 7-day dengue prediction
    
    Parameters:
    - df: DataFrame with historical dengue cases
    - target_location: ISO country code
    - prediction_date: Date to predict from (default: today)
    """
    from datetime import datetime, timedelta
    
    if prediction_date is None:
        prediction_date = datetime.now()
    else:
        prediction_date = pd.to_datetime(prediction_date)
    
    # Filter data for target location
    location_data = df[df['ISO_A0'] == target_location].copy()
    location_data = location_data.sort_values('calendar_start_date')
    
    # Get most recent data point
    latest_date = location_data['calendar_start_date'].max()
    latest_cases = location_data[location_data['calendar_start_date'] == latest_date]['dengue_total'].values
    
    if len(latest_cases) == 0:
        print(f"No data found for {target_location}")
        return None
    
    # Create feature dictionary
    features = {
        # Historical case features
        'cases_last_7d': location_data[
            (location_data['calendar_start_date'] >= prediction_date - timedelta(days=7)) &
            (location_data['calendar_start_date'] < prediction_date)
        ]['dengue_total'].sum(),
        
        'cases_last_30d': location_data[
            (location_data['calendar_start_date'] >= prediction_date - timedelta(days=30)) &
            (location_data['calendar_start_date'] < prediction_date)
        ]['dengue_total'].sum(),
        
        'cases_last_90d': location_data[
            (location_data['calendar_start_date'] >= prediction_date - timedelta(days=90)) &
            (location_data['calendar_start_date'] < prediction_date)
        ]['dengue_total'].sum(),
        
        # Temporal features
        'month': prediction_date.month,
        'day_of_year': prediction_date.timetuple().tm_yday,
        'quarter': prediction_date.quarter,
        
        # Seasonal average (historical cases for same month)
        'historical_monthly_avg': location_data[
            location_data['calendar_start_date'].dt.month == prediction_date.month
        ]['dengue_total'].mean(),
        
        # Trend
        'recent_trend': None,  # Will calculate from recent data
    }
    
    # Calculate trend
    recent_7d = features['cases_last_7d']
    previous_7d = location_data[
        (location_data['calendar_start_date'] >= prediction_date - timedelta(days=14)) &
        (location_data['calendar_start_date'] < prediction_date - timedelta(days=7))
    ]['dengue_total'].sum()
    
    if previous_7d > 0:
        features['recent_trend'] = (recent_7d - previous_7d) / previous_7d
    else:
        features['recent_trend'] = 0
    
    # Note: Weather features need to be added from external API
    # features['temperature'] = ...
    # features['rainfall'] = ...
    # features['humidity'] = ...
    
    return features

# Example usage
example_features = prepare_features_for_prediction(df, target_location='BGD', 
                                                   prediction_date='2024-01-01')
print("Example features for prediction:")
for key, value in example_features.items():
    print(f"  {key}: {value}")


Example features for prediction:
  cases_last_7d: 0
  cases_last_30d: 0
  cases_last_90d: 53499
  month: 1
  day_of_year: 1
  quarter: 1
  historical_monthly_avg: 617.8888888888889
  recent_trend: 0


## Recommended Feature Priority for 7-Day Prediction

### **High Priority (Must Have):**
1. **Recent Case History** (Last 7, 14, 30 days) - From your existing data ✅
2. **Temperature** (Current + Forecast) - Weather API
3. **Rainfall** (Current + Forecast) - Weather API
4. **Seasonal Patterns** (Month, historical averages) - From your existing data ✅

### **Medium Priority (Should Have):**
5. **Humidity** - Weather API
6. **Case Trends** (Increasing/Decreasing) - Calculate from existing data ✅
7. **Historical Seasonal Averages** - From your existing data ✅

### **Low Priority (Nice to Have):**
8. **Population Density** - World Bank/UN data
9. **Previous Outbreak Intensity** - From your existing data ✅
10. **Mosquito Indices** - If available from health departments

## Next Steps:

1. **Collect Weather Data**: Set up API access to OpenWeatherMap or similar
2. **Feature Engineering**: Create lag features, rolling averages, trends
3. **Model Selection**: Time series models (ARIMA, Prophet) or ML models (XGBoost, Random Forest)
4. **Training**: Use historical data with weather features
5. **Prediction**: Generate 7-day forecasts with current weather forecasts


In [28]:
# Quick analysis: What data do we have available?
print("=== Data Availability Analysis ===\n")

print(f"Total records: {len(df):,}")
print(f"Date range: {df['calendar_start_date'].min()} to {df['calendar_start_date'].max()}")
print(f"Countries: {df['ISO_A0'].nunique()}")
print(f"Regions: {df['region'].nunique()}")

print("\n=== Countries in dataset ===")
print(df['ISO_A0'].value_counts().head(20))

print("\n=== Temporal resolution ===")
# Check if we have daily, weekly, or monthly data
df['date_diff'] = (df['calendar_end_date'] - df['calendar_start_date']).dt.days
print(f"Average period length: {df['date_diff'].mean():.1f} days")
print(f"Period types:")
print(df['date_diff'].value_counts().head(10))

print("\n=== Case statistics ===")
print(f"Total cases: {df['dengue_total'].sum():,}")
print(f"Average cases per period: {df['dengue_total'].mean():.2f}")
print(f"Max cases in single period: {df['dengue_total'].max():,}")

# Check for Bangladesh specifically
bd_data = df[df['ISO_A0'] == 'BGD']
if len(bd_data) > 0:
    print("\n=== Bangladesh-specific ===")
    print(f"Records: {len(bd_data)}")
    print(f"Date range: {bd_data['calendar_start_date'].min()} to {bd_data['calendar_start_date'].max()}")
    print(f"Total cases: {bd_data['dengue_total'].sum():,}")
    print(f"Average cases: {bd_data['dengue_total'].mean():.2f}")


=== Data Availability Analysis ===

Total records: 41,997
Date range: 1958-01-01 00:00:00 to 2025-03-01 00:00:00
Countries: 10
Regions: 1

=== Countries in dataset ===
ISO_A0
THA    27259
IDN     8396
NPL     2983
LKA     1718
IND      705
MMR      262
BGD      234
MDV      211
TLS      160
BTN       69
Name: count, dtype: int64

=== Temporal resolution ===
Average period length: 44.4 days
Period types:
date_diff
30     23017
29     13051
27      2495
364     1386
28       824
6        682
365      542
Name: count, dtype: int64

=== Case statistics ===
Total cases: 18,133,941
Average cases per period: 431.79
Max cases in single period: 174,285

=== Bangladesh-specific ===
Records: 234
Date range: 1980-01-01 00:00:00 to 2025-03-01 00:00:00
Total cases: 672,026
Average cases: 2871.91


In [29]:
# Complete Feature Set Template for 7-Day Prediction
# This combines all features needed for prediction

def create_complete_feature_set(df, location_code, location_name, lat, lon, 
                                prediction_date=None, weather_api_key=None):
    """
    Create complete feature set for 7-day dengue prediction
    
    Returns a dictionary with all features needed for prediction
    """
    from datetime import datetime, timedelta
    
    if prediction_date is None:
        prediction_date = datetime.now()
    else:
        prediction_date = pd.to_datetime(prediction_date)
    
    # 1. Historical case features (from existing data)
    location_data = df[df['ISO_A0'] == location_code].copy()
    location_data = location_data.sort_values('calendar_start_date')
    
    features = {}
    
    # Case history features
    for days in [7, 14, 21, 30, 60, 90]:
        mask = (location_data['calendar_start_date'] >= prediction_date - timedelta(days=days)) & \
               (location_data['calendar_start_date'] < prediction_date)
        features[f'cases_last_{days}d'] = location_data[mask]['dengue_total'].sum()
    
    # Rolling averages
    recent_data = location_data[location_data['calendar_start_date'] < prediction_date].tail(30)
    if len(recent_data) > 0:
        features['cases_7d_rolling_mean'] = recent_data.tail(7)['dengue_total'].mean()
        features['cases_30d_rolling_mean'] = recent_data['dengue_total'].mean()
    else:
        features['cases_7d_rolling_mean'] = 0
        features['cases_30d_rolling_mean'] = 0
    
    # Trend calculation
    week1 = features['cases_last_7d']
    week2 = location_data[
        (location_data['calendar_start_date'] >= prediction_date - timedelta(days=14)) &
        (location_data['calendar_start_date'] < prediction_date - timedelta(days=7))
    ]['dengue_total'].sum()
    features['case_trend'] = (week1 - week2) / max(week2, 1)  # Percentage change
    
    # 2. Temporal features
    features['month'] = prediction_date.month
    features['day_of_year'] = prediction_date.timetuple().tm_yday
    features['quarter'] = prediction_date.quarter
    features['is_rainy_season'] = 1 if prediction_date.month in [6, 7, 8, 9, 10] else 0  # Adjust for your region
    
    # Historical seasonal average
    same_month_data = location_data[location_data['calendar_start_date'].dt.month == prediction_date.month]
    features['historical_monthly_avg'] = same_month_data['dengue_total'].mean() if len(same_month_data) > 0 else 0
    
    # 3. Weather features (placeholder - needs API integration)
    if weather_api_key:
        try:
            weather = fetch_weather_data_owm(location_name, lat, lon, weather_api_key, None, None)
            if weather:
                features.update({
                    'temperature': weather.get('temperature', None),
                    'temp_min': weather.get('temp_min', None),
                    'temp_max': weather.get('temp_max', None),
                    'humidity': weather.get('humidity', None),
                    'rainfall': weather.get('rainfall', 0),
                    'wind_speed': weather.get('wind_speed', None)
                })
        except:
            pass
    
    # 4. Derived weather features (if temperature available)
    if 'temperature' in features and features['temperature']:
        # Optimal temperature range for dengue: 20-30°C
        temp = features['temperature']
        features['temp_optimal'] = 1 if 20 <= temp <= 30 else 0
        features['temp_too_cold'] = 1 if temp < 20 else 0
        features['temp_too_hot'] = 1 if temp > 30 else 0
    
    # 5. Location features
    features['location_code'] = location_code
    features['region'] = location_data['region'].iloc[0] if len(location_data) > 0 else None
    
    return features

# Example: Create feature set for Bangladesh (Dhaka coordinates)
# Note: You'll need to provide weather API key for weather features
example_features = create_complete_feature_set(
    df, 
    location_code='BGD',
    location_name='Dhaka',
    lat=23.8103,
    lon=90.4125,
    prediction_date='2024-01-01',
    weather_api_key=None  # Add your API key here
)

print("Complete feature set example:")
print("=" * 50)
for key, value in sorted(example_features.items()):
    print(f"{key:30s}: {value}")


Complete feature set example:
case_trend                    : 0.0
cases_30d_rolling_mean        : 13644.166666666666
cases_7d_rolling_mean         : 45629.71428571428
cases_last_14d                : 0
cases_last_21d                : 0
cases_last_30d                : 0
cases_last_60d                : 7473
cases_last_7d                 : 0
cases_last_90d                : 53499
day_of_year                   : 1
historical_monthly_avg        : 617.8888888888889
is_rainy_season               : 0
location_code                 : BGD
month                         : 1
quarter                       : 1
region                        : SEARO


## Summary: Feature Collection Checklist

### ✅ Features You Can Extract NOW (from existing data):
- [x] Historical case counts (last 7, 14, 30, 60, 90 days)
- [x] Rolling averages and trends
- [x] Seasonal patterns (month, quarter, day of year)
- [x] Historical monthly averages
- [x] Location/region information

### ⚠️ Features You Need to Collect (external sources):

#### Weather Data (CRITICAL):
1. **OpenWeatherMap API** (Recommended for beginners)
   - Sign up: https://openweathermap.org/api
   - Free tier: 60 calls/minute, current + 5-day forecast
   - Cost: Free for basic use
   - Get API key and use the `fetch_weather_data_owm()` function above

2. **Alternative: WeatherAPI.com**
   - Free tier: 1 million calls/month
   - Historical data available
   - https://www.weatherapi.com/

#### Population/Immunity Data (Optional but helpful):
1. **World Bank Data API** (Free)
   - Population density
   - https://datahelpdesk.worldbank.org/knowledgebase/articles/889392

2. **WHO Data** (Free)
   - Health statistics
   - https://www.who.int/data/gho

### 📝 Action Items:

1. **Immediate (Can do now):**
   - Run the feature engineering code above
   - Create lag features from your existing data
   - Calculate trends and seasonal patterns

2. **Next Step (This week):**
   - Sign up for OpenWeatherMap API (free)
   - Get coordinates (lat/lon) for your target locations
   - Integrate weather data collection

3. **For Production:**
   - Set up automated daily data collection
   - Create feature pipeline
   - Train prediction model (XGBoost, Random Forest, or LSTM)
   - Deploy prediction system

### 🔑 Key Insight:
For 7-day prediction, **weather forecast data is essential** because:
- Mosquito breeding cycles are 7-10 days
- Temperature affects virus replication (2-3 days)
- Rainfall creates breeding sites (effect seen 2-4 weeks later)

**Priority order:**
1. Recent case history (you have this ✅)
2. Weather forecast (need API ⚠️)
3. Seasonal patterns (you have this ✅)
4. Historical trends (you have this ✅)


## Quick Reference: Feature Matrix

| Feature | Priority | Source | Status | Notes |
|---------|----------|--------|--------|-------|
| **Cases (last 7d)** | 🔴 High | Your data | ✅ Available | Most important predictor |
| **Cases (last 14d)** | 🔴 High | Your data | ✅ Available | Short-term trend |
| **Cases (last 30d)** | 🔴 High | Your data | ✅ Available | Medium-term baseline |
| **Temperature** | 🔴 High | Weather API | ⚠️ Need to collect | Optimal: 20-30°C |
| **Rainfall** | 🔴 High | Weather API | ⚠️ Need to collect | Lag effect: 2-4 weeks |
| **Case Trend** | 🟡 Medium | Your data | ✅ Can calculate | Increasing/decreasing |
| **Seasonal Pattern** | 🟡 Medium | Your data | ✅ Available | Month, day of year |
| **Humidity** | 🟡 Medium | Weather API | ⚠️ Need to collect | Affects mosquito survival |
| **Historical Avg** | 🟡 Medium | Your data | ✅ Available | Same month previous years |
| **Population Density** | 🟢 Low | World Bank | ⚠️ Optional | Urban vs rural |
| **Immunity Proxy** | 🟢 Low | Your data | ✅ Can calculate | Cases in last 6-12 months |
| **Wind Speed** | 🟢 Low | Weather API | ⚠️ Optional | Mosquito dispersal |

### Feature Importance for 7-Day Prediction:

1. **Recent Cases (7-30 days)** - Direct indicator of current outbreak status
2. **Weather Forecast** - Critical for predicting mosquito breeding conditions
3. **Seasonal Patterns** - Captures annual cycles
4. **Trends** - Indicates if outbreak is growing or declining

### Minimum Viable Feature Set:
If you can only collect a few features, prioritize:
1. Cases last 7 days
2. Cases last 30 days  
3. Temperature (current + 7-day forecast)
4. Rainfall (current + 7-day forecast)
5. Month/Season


In [30]:
# ============================================
# COMPREHENSIVE FEATURE ENGINEERING PIPELINE
# Creating final_data.csv with all merged features
# ============================================

print("Starting feature engineering pipeline...")
print("=" * 60)

# Step 1: Load and prepare base data
print("\n[1/7] Loading and preparing base data...")
df_base = pd.read_csv("dengu_out_break.csv", low_memory=False)

# Convert dates
df_base['calendar_start_date'] = pd.to_datetime(df_base['calendar_start_date'])
df_base['calendar_end_date'] = pd.to_datetime(df_base['calendar_end_date'])

# Create a unique location identifier
df_base['location_id'] = df_base['ISO_A0'].astype(str) + '_' + \
                         df_base['adm_1_name'].fillna('').astype(str) + '_' + \
                         df_base['adm_2_name'].fillna('').astype(str)

# Sort by location and date
df_base = df_base.sort_values(['location_id', 'calendar_start_date']).reset_index(drop=True)

print(f"  ✓ Loaded {len(df_base):,} records")
print(f"  ✓ Date range: {df_base['calendar_start_date'].min()} to {df_base['calendar_start_date'].max()}")
print(f"  ✓ Unique locations: {df_base['location_id'].nunique()}")


Starting feature engineering pipeline...

[1/7] Loading and preparing base data...
  ✓ Loaded 41,997 records
  ✓ Date range: 1958-01-01 00:00:00 to 2025-03-01 00:00:00
  ✓ Unique locations: 372


In [31]:
# Step 2: Create temporal features
print("\n[2/7] Creating temporal features...")

df_features = df_base.copy()

# Basic temporal features
df_features['year'] = df_features['calendar_start_date'].dt.year
df_features['month'] = df_features['calendar_start_date'].dt.month
df_features['day'] = df_features['calendar_start_date'].dt.day
df_features['day_of_year'] = df_features['calendar_start_date'].dt.dayofyear
df_features['quarter'] = df_features['calendar_start_date'].dt.quarter
df_features['week'] = df_features['calendar_start_date'].dt.isocalendar().week
df_features['day_of_week'] = df_features['calendar_start_date'].dt.dayofweek
df_features['is_weekend'] = (df_features['day_of_week'] >= 5).astype(int)

# Period length (days between start and end)
df_features['period_length'] = (df_features['calendar_end_date'] - df_features['calendar_start_date']).dt.days + 1

# Seasonal indicators (adjust based on your region - this is for tropical regions)
df_features['is_rainy_season'] = df_features['month'].isin([6, 7, 8, 9, 10]).astype(int)
df_features['is_dry_season'] = (~df_features['month'].isin([6, 7, 8, 9, 10])).astype(int)

# Cyclical encoding for month (sine/cosine)
df_features['month_sin'] = np.sin(2 * np.pi * df_features['month'] / 12)
df_features['month_cos'] = np.cos(2 * np.pi * df_features['month'] / 12)
df_features['day_of_year_sin'] = np.sin(2 * np.pi * df_features['day_of_year'] / 365)
df_features['day_of_year_cos'] = np.cos(2 * np.pi * df_features['day_of_year'] / 365)

print(f"  ✓ Created {len([c for c in df_features.columns if c not in df_base.columns])} temporal features")



[2/7] Creating temporal features...
  ✓ Created 15 temporal features


In [32]:
# Step 3: Create lag features (cases from previous periods)
print("\n[3/7] Creating lag features...")

# Group by location for lag calculations
lag_periods = [1, 2, 3, 4, 5, 6, 7, 14, 21, 30, 60, 90, 180, 365]

for lag in lag_periods:
    df_features[f'cases_lag_{lag}d'] = df_features.groupby('location_id')['dengue_total'].shift(lag)
    # Fill NaN with 0 for locations without enough history
    df_features[f'cases_lag_{lag}d'] = df_features[f'cases_lag_{lag}d'].fillna(0)

print(f"  ✓ Created {len(lag_periods)} lag features")



[3/7] Creating lag features...
  ✓ Created 14 lag features


In [33]:
# Step 4: Create rolling statistics
print("\n[4/7] Creating rolling statistics...")

rolling_windows = [7, 14, 21, 30, 60, 90, 180, 365]

for window in rolling_windows:
    # Rolling mean
    df_features[f'cases_rolling_mean_{window}d'] = df_features.groupby('location_id')['dengue_total'].transform(
        lambda x: x.shift(1).rolling(window=window, min_periods=1).mean()
    )
    
    # Rolling sum
    df_features[f'cases_rolling_sum_{window}d'] = df_features.groupby('location_id')['dengue_total'].transform(
        lambda x: x.shift(1).rolling(window=window, min_periods=1).sum()
    )
    
    # Rolling max
    df_features[f'cases_rolling_max_{window}d'] = df_features.groupby('location_id')['dengue_total'].transform(
        lambda x: x.shift(1).rolling(window=window, min_periods=1).max()
    )
    
    # Rolling std (variability)
    df_features[f'cases_rolling_std_{window}d'] = df_features.groupby('location_id')['dengue_total'].transform(
        lambda x: x.shift(1).rolling(window=window, min_periods=1).std().fillna(0)
    )

print(f"  ✓ Created {len(rolling_windows) * 4} rolling statistics features")



[4/7] Creating rolling statistics...
  ✓ Created 32 rolling statistics features


In [34]:
# Step 5: Create trend and change features
print("\n[5/7] Creating trend and change features...")

# Week-over-week change
df_features['cases_wow_change'] = df_features.groupby('location_id')['dengue_total'].pct_change(periods=1).fillna(0)
df_features['cases_wow_abs_change'] = df_features.groupby('location_id')['dengue_total'].diff(periods=1).fillna(0)

# Compare recent periods
df_features['cases_7d_vs_30d'] = (df_features['cases_rolling_mean_7d'] - df_features['cases_rolling_mean_30d']).fillna(0)
df_features['cases_7d_vs_30d_pct'] = (
    (df_features['cases_rolling_mean_7d'] - df_features['cases_rolling_mean_30d']) / 
    (df_features['cases_rolling_mean_30d'] + 1) * 100
).fillna(0)

df_features['cases_30d_vs_90d'] = (df_features['cases_rolling_mean_30d'] - df_features['cases_rolling_mean_90d']).fillna(0)
df_features['cases_30d_vs_90d_pct'] = (
    (df_features['cases_rolling_mean_30d'] - df_features['cases_rolling_mean_90d']) / 
    (df_features['cases_rolling_mean_90d'] + 1) * 100
).fillna(0)

# Acceleration (rate of change of cases)
df_features['cases_acceleration'] = df_features.groupby('location_id')['dengue_total'].diff(periods=2).fillna(0)

print(f"  ✓ Created trend and change features")



[5/7] Creating trend and change features...
  ✓ Created trend and change features


In [35]:
# Step 6: Create historical seasonal features
print("\n[6/7] Creating historical seasonal features...")

# Historical average for same month (across all years)
df_features['historical_monthly_avg'] = df_features.groupby(['location_id', 'month'])['dengue_total'].transform(
    lambda x: x.shift(1).expanding().mean()
).fillna(0)

# Historical average for same quarter
df_features['historical_quarterly_avg'] = df_features.groupby(['location_id', 'quarter'])['dengue_total'].transform(
    lambda x: x.shift(1).expanding().mean()
).fillna(0)

# Historical max for same month
df_features['historical_monthly_max'] = df_features.groupby(['location_id', 'month'])['dengue_total'].transform(
    lambda x: x.shift(1).expanding().max()
).fillna(0)

# Cases in same month previous year
df_features['cases_same_month_prev_year'] = df_features.groupby('location_id').apply(
    lambda x: x.set_index('calendar_start_date')['dengue_total'].shift(365)
).reset_index(level=0, drop=True).reset_index(drop=True).fillna(0)

# Cases in same quarter previous year
df_features['cases_same_quarter_prev_year'] = df_features.groupby('location_id').apply(
    lambda x: x.set_index('calendar_start_date')['dengue_total'].shift(365)
).reset_index(level=0, drop=True).reset_index(drop=True).fillna(0)

# Ratio to historical average
df_features['cases_to_historical_monthly_avg'] = (
    df_features['dengue_total'] / (df_features['historical_monthly_avg'] + 1)
).fillna(0)

print(f"  ✓ Created historical seasonal features")



[6/7] Creating historical seasonal features...


/tmp/ipykernel_8406/2294262089.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_features['cases_same_month_prev_year'] = df_features.groupby('location_id').apply(


  ✓ Created historical seasonal features


/tmp/ipykernel_8406/2294262089.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_features['cases_same_quarter_prev_year'] = df_features.groupby('location_id').apply(


In [36]:
# Step 7: Create location-based aggregated features
print("\n[7/7] Creating location-based aggregated features...")

# Total cases in location (cumulative)
df_features['location_total_cases'] = df_features.groupby('location_id')['dengue_total'].cumsum()

# Average cases per period for location
df_features['location_avg_cases'] = df_features.groupby('location_id')['dengue_total'].transform(
    lambda x: x.expanding().mean()
)

# Location outbreak intensity (cases per period length)
df_features['cases_per_day'] = df_features['dengue_total'] / (df_features['period_length'] + 1)

# Region-level features
df_features['region_total_cases'] = df_features.groupby(['region', 'calendar_start_date'])['dengue_total'].transform('sum')
df_features['region_avg_cases'] = df_features.groupby('region')['dengue_total'].transform('mean')

# Country-level features
df_features['country_total_cases'] = df_features.groupby(['ISO_A0', 'calendar_start_date'])['dengue_total'].transform('sum')
df_features['country_avg_cases'] = df_features.groupby('ISO_A0')['dengue_total'].transform('mean')

print(f"  ✓ Created location-based aggregated features")



[7/7] Creating location-based aggregated features...
  ✓ Created location-based aggregated features


In [37]:
# Step 8: Final cleanup and save
print("\n" + "=" * 60)
print("Finalizing dataset...")

# Remove rows with all NaN in key features (if any)
initial_rows = len(df_features)
df_features = df_features.dropna(subset=['dengue_total', 'calendar_start_date'])
final_rows = len(df_features)

if initial_rows != final_rows:
    print(f"  ⚠ Removed {initial_rows - final_rows} rows with missing critical data")

# Fill remaining NaN values with 0 for numeric features (except original columns)
numeric_cols = df_features.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if col not in ['dengue_total', 'Year', 'FAO_GAUL_code']:  # Don't fill target or original columns
        df_features[col] = df_features[col].fillna(0)

# Sort final dataset
df_features = df_features.sort_values(['location_id', 'calendar_start_date']).reset_index(drop=True)

# Display summary
print(f"\n✓ Final dataset shape: {df_features.shape}")
print(f"✓ Total features: {len(df_features.columns)}")
print(f"✓ Total records: {len(df_features):,}")

print("\n" + "=" * 60)
print("Feature categories created:")
print(f"  - Original columns: {len(df_base.columns)}")
print(f"  - Temporal features: ~15")
print(f"  - Lag features: {len(lag_periods)}")
print(f"  - Rolling statistics: {len(rolling_windows) * 4}")
print(f"  - Trend features: ~8")
print(f"  - Historical seasonal: ~6")
print(f"  - Location aggregated: ~8")
print(f"  - Total new features: ~{len(df_features.columns) - len(df_base.columns)}")

# Save to CSV
output_file = "final_data.csv"
print(f"\n💾 Saving to {output_file}...")
df_features.to_csv(output_file, index=False)
print(f"✅ Successfully saved {len(df_features):,} records with {len(df_features.columns)} features to {output_file}")

# Display first few rows
print("\n" + "=" * 60)
print("Sample of final dataset:")
print(df_features[['ISO_A0', 'adm_1_name', 'calendar_start_date', 'dengue_total', 
                   'cases_lag_7d', 'cases_rolling_mean_7d', 'cases_rolling_mean_30d', 
                   'month', 'historical_monthly_avg']].head(10))



Finalizing dataset...

✓ Final dataset shape: (41997, 99)
✓ Total features: 99
✓ Total records: 41,997

Feature categories created:
  - Original columns: 18
  - Temporal features: ~15
  - Lag features: 14
  - Rolling statistics: 32
  - Trend features: ~8
  - Historical seasonal: ~6
  - Location aggregated: ~8
  - Total new features: ~81

💾 Saving to final_data.csv...
✅ Successfully saved 41,997 records with 99 features to final_data.csv

Sample of final dataset:
  ISO_A0 adm_1_name calendar_start_date  dengue_total  cases_lag_7d  \
0    BGD        NaN          1980-01-01             4           0.0   
1    BGD        NaN          1985-01-01             0           0.0   
2    BGD        NaN          1986-01-01             0           0.0   
3    BGD        NaN          1987-01-01             0           0.0   
4    BGD        NaN          1988-01-01             0           0.0   
5    BGD        NaN          1989-01-01             0           0.0   
6    BGD        NaN          1990-0

In [38]:
# Display feature summary
print("\n" + "=" * 60)
print("FEATURE SUMMARY")
print("=" * 60)

print("\n📊 All Features in final_data.csv:")
print(f"\nTotal columns: {len(df_features.columns)}")

# Group features by category
feature_categories = {
    'Original': ['adm_0_name', 'adm_1_name', 'adm_2_name', 'full_name', 'ISO_A0', 
                'FAO_GAUL_code', 'RNE_iso_code', 'calendar_start_date', 'calendar_end_date',
                'Year', 'dengue_total', 'case_definition_standardised', 'S_res', 'T_res', 
                'UUID', 'region'],
    'Temporal': [col for col in df_features.columns if any(x in col for x in ['year', 'month', 'day', 'quarter', 'week', 'season', 'sin', 'cos'])],
    'Lag Features': [col for col in df_features.columns if 'lag' in col],
    'Rolling Statistics': [col for col in df_features.columns if 'rolling' in col],
    'Trend/Change': [col for col in df_features.columns if any(x in col for x in ['trend', 'change', 'wow', 'vs_', 'acceleration'])],
    'Historical': [col for col in df_features.columns if 'historical' in col or 'prev_year' in col],
    'Location Aggregated': [col for col in df_features.columns if any(x in col for x in ['location_', 'region_', 'country_', 'cases_per_day'])]
}

for category, features in feature_categories.items():
    if features:
        print(f"\n{category} ({len(features)} features):")
        for feat in sorted(features)[:10]:  # Show first 10
            print(f"  - {feat}")
        if len(features) > 10:
            print(f"  ... and {len(features) - 10} more")

print("\n" + "=" * 60)
print("✅ Feature engineering complete!")
print("=" * 60)



FEATURE SUMMARY

📊 All Features in final_data.csv:

Total columns: 99

Original (16 features):
  - FAO_GAUL_code
  - ISO_A0
  - RNE_iso_code
  - S_res
  - T_res
  - UUID
  - Year
  - adm_0_name
  - adm_1_name
  - adm_2_name
  ... and 6 more

Temporal (21 features):
  - cases_per_day
  - cases_same_month_prev_year
  - cases_same_quarter_prev_year
  - cases_to_historical_monthly_avg
  - day
  - day_of_week
  - day_of_year
  - day_of_year_cos
  - day_of_year_sin
  - historical_monthly_avg
  ... and 11 more

Lag Features (14 features):
  - cases_lag_14d
  - cases_lag_180d
  - cases_lag_1d
  - cases_lag_21d
  - cases_lag_2d
  - cases_lag_30d
  - cases_lag_365d
  - cases_lag_3d
  - cases_lag_4d
  - cases_lag_5d
  ... and 4 more

Rolling Statistics (32 features):
  - cases_rolling_max_14d
  - cases_rolling_max_180d
  - cases_rolling_max_21d
  - cases_rolling_max_30d
  - cases_rolling_max_365d
  - cases_rolling_max_60d
  - cases_rolling_max_7d
  - cases_rolling_max_90d
  - cases_rolling_mean_